In [8]:
%load_ext autoreload
%autoreload

import deep_nmmo
from deep_nmmo.envs.team_based_env.teams.custom_team import CustomTeam
from deep_nmmo.utils import get_class_from_path, get_module_from_path
from deep_nmmo.envs.team_based_env.env_configs.custom_competition_config import CustomCompetitionConfig
from deep_nmmo.envs.team_based_env.loops.utils import init_env_params, reset_teams, reset_env

import nmmo
from nmmo import config
from nmmo.io import action
from nmmo import scripting, material, Serialized
from nmmo.systems import skill, item
from nmmo.lib import colors
from nmmo import action as Action

import neurips2022nmmo
from neurips2022nmmo.scripted import baselines
from neurips2022nmmo import Team
from neurips2022nmmo import CompetitionConfig, scripted, RollOut, TeamBasedEnv
from neurips2022nmmo.scripted import attack, move

import ray
ray.shutdown()
ray.init()

from ray.rllib.agents import ppo
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

# from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray.rllib.policy.policy import PolicySpec

from ray.rllib.models.modelv2 import restore_original_dimensions
from collections.abc import Mapping

from ray.rllib.utils.annotations import override
from ray.rllib.utils.typing import (
    # AgentID,
    # EnvCreator,
    # EnvID,
    # EnvType,
    MultiAgentDict,
    # MultiEnvDict,
)

import gym

import numpy as np

import copy
from collections import defaultdict

from IPython.display import display

from typing import Dict, Any, Type, List

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2022-09-29 20:42:57,513	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8269 


In [13]:
%autoreload

from deep_nmmo.envs.rllib_multi_agent_team_based_env.env import RLlibMultiAgentTeamBasedEnv
from deep_nmmo.envs.rllib_multi_agent_team_based_env.agents.rllib_random_policy import RLlibRandomPolicy


path_to_rllib_trainer_cls = 'ray.rllib.agents.ppo.PPOTrainer'

path_to_env_config_cls = 'deep_nmmo.envs.team_based_env.env_configs.custom_competition_config.CustomCompetitionConfig'

num_players_per_team = 8
# will overwrite agents kwargs with observation and action space and player index when initialising teams in RLlib env
custom_team_kwargs = {'agents_cls': [RLlibRandomPolicy for _ in range(num_players_per_team)], 
                      'agents_kwargs': [None for _ in range(num_players_per_team)],
                     }

teams_config = {
    # 'RLlib': {'cls': RLlibScriptedHybridAgentTeam, 'kwargs': {'paths_to_scripted_agents_cls': rllib_paths_to_scripted_agents_cls}},
    
    # 'Combat-1': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 1}},
    # 'Combat-2': {'cls': neurips2022nmmo.scripted.CombatTeam, 'kwargs': {'team_id': 2}},
    # 'Mixture-1': {'cls': neurips2022nmmo.scripted.MixtureTeam, 'kwargs': {'team_id': 3}},
    
    'R-1': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-2': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    'R-3': {'cls': deep_nmmo.envs.team_based_env.teams.custom_team.CustomTeam, 'kwargs': custom_team_kwargs},
    
    
}

env_config = get_class_from_path(path_to_env_config_cls)()

policies = {}
policy_id = 1
num_teams, num_players_per_team = 3, 8
for _ in range(num_teams):
    player_idx = 0
    for _ in range(num_players_per_team):
        policies[str(policy_id)] = PolicySpec(
                                    policy_class=RLlibRandomPolicy, # infer from env
                                    # observation_space=dummy_env.observation_space(player_id),
                                    # action_space=dummy_env.action_space(player_id),
                                    config={'env_config': env_config, 'idx': player_idx},
                                )
        policy_id += 1
        player_idx += 1

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    '''Maps agent ID (int) to corresponding policy ID (str) of policy which agent should use.'''
    print(f'>>> Mapping agent_id {agent_id} to {str(agent_id)}')
    return str(agent_id)

multiagent_config = {
    'policies': policies,
    'policy_mapping_fn': policy_mapping_fn,
    'policies_to_train': [],
}

env_config = {
    # 'path_to_env_cls': path_to_env_cls,
    # 'path_to_env_config_cls': path_to_env_config_cls,
    'env_config': env_config,
    'teams_config': teams_config,
}


rllib_config = {
    'env': 'ma_env',
    
    'env_config': env_config,
    
    'disable_env_checking': True,
    # 'disable_env_checking': False,
    
    'framework': 'torch',
    
    'multiagent': multiagent_config,
    
    'num_workers': 0,
    
}

In [14]:
%autoreload

# register env with ray
register_env('ma_env', lambda env_config: RLlibMultiAgentTeamBasedEnv(**env_config))

In [15]:
%autoreload

# merge rllibg trainer's default config with specified config
path_to_agent = '.'.join(path_to_rllib_trainer_cls.split('.')[:-1])
_rllib_config = get_module_from_path(path_to_agent).DEFAULT_CONFIG.copy()
_rllib_config.update(rllib_config)
# print(_rllib_config)

# init rllib trainer
trainer = get_class_from_path(path_to_rllib_trainer_cls)(config=_rllib_config)
print(f'Initialised trainer')

2022-09-29 20:58:59,004	WARNING multi_agent.py:121 -- `config.multiagent.policies_to_train` is empty! Make sure - if you would like to learn at least one policy - to add its ID to that list.
  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 13.15it/s]

Generating 40 maps


  5%|████████▎                                                                                                                                                              | 2/40 [00:00<00:02, 13.44it/s]

Generating 40 maps


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:02<00:00, 13.93it/s]
2022-09-29 20:59:07,151	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Initialised trainer


In [16]:
%autoreload

# perform one training epoch
results = trainer.train()
print(f'Completed RLlib epoch!!!')


~~~ RESET ~~~
RLlibMultiAgentTeamBasedEnv obs keys: 24 dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
>>> Mapping agent_id 1 to 1
>>> Mapping agent_id 2 to 2
>>> Mapping agent_id 3 to 3
>>> Mapping agent_id 4 to 4
>>> Mapping agent_id 5 to 5
>>> Mapping agent_id 6 to 6
>>> Mapping agent_id 7 to 7
>>> Mapping agent_id 8 to 8
>>> Mapping agent_id 9 to 9
>>> Mapping agent_id 10 to 10
>>> Mapping agent_id 11 to 11
>>> Mapping agent_id 12 to 12
>>> Mapping agent_id 13 to 13
>>> Mapping agent_id 14 to 14
>>> Mapping agent_id 15 to 15
>>> Mapping agent_id 16 to 16
>>> Mapping agent_id 17 to 17
>>> Mapping agent_id 18 to 18
>>> Mapping agent_id 19 to 19
>>> Mapping agent_id 20 to 20
>>> Mapping agent_id 21 to 21
>>> Mapping agent_id 22 to 22
>>> Mapping agent_id 23 to 23
>>> Mapping agent_id 24 to 24

~~~ Step 1 ~~~
action_dict keys: 24 dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
RLlibMu

ValueError: all input arrays must have the same shape

In [ ]:
import copy